In [ ]:
import os
import numpy as np
from skimage.io import imread
from skimage.color import rgb2gray
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

# Define folders
ref_root = 'DATASET'  
test_root = 'DATASET_gaussian'       

# Storage
ssim_scores = []
psnr_scores = []

# Walk through all subfolders in reference
for dirpath, _, filenames in os.walk(ref_root):
    filenames.sort()
    for fname in filenames:
        rel_path = os.path.relpath(os.path.join(dirpath, fname), ref_root)
        ref_path = os.path.join(ref_root, rel_path)
        test_path = os.path.join(test_root, rel_path)

        if not os.path.exists(test_path):
            continue

        try:
            ref_img = imread(ref_path)
            test_img = imread(test_path)

            if ref_img.shape != test_img.shape:
                continue

            # Convert to grayscale if RGB
            if ref_img.ndim == 3 and ref_img.shape[2] == 3:
                ref_img = rgb2gray(ref_img)
                test_img = rgb2gray(test_img)
                data_range = 1.0
            else:
                ref_img = ref_img.astype(np.float32)
                test_img = test_img.astype(np.float32)
                range_val = test_img.max() - test_img.min()
                data_range = range_val if range_val > 0 else 1.0

            # Compute SSIM (unitless)
            ssim_val = ssim(ref_img, test_img, data_range=data_range)
            # Convert SSIM to percentage
            ssim_pct = ssim_val * 100.0

            # Compute PSNR (dB)
            psnr_val = psnr(ref_img, test_img, data_range=data_range)

            # Store
            ssim_scores.append(ssim_pct)
            psnr_scores.append(psnr_val)

        except Exception:
            continue

# Final results
if ssim_scores:
    mean_ssim_pct = np.mean(ssim_scores)
    mean_psnr = np.mean(psnr_scores)

    print("\n========== SUMMARY ==========")
    print(f"Total images processed: {len(ssim_scores)}")
    print(f"Mean SSIM:    {mean_ssim_pct:.2f} %")
    print(f"Mean PSNR:    {mean_psnr:.2f} dB")
else:
    print("No valid image pairs found.")



========== SUMMARY ==========
Total images processed: 25
Mean SSIM:    87.65 %
Mean PSNR:    25.52 dB
